In [1]:
from __future__ import print_function
import sys
import os
import argparse
import datetime
import numpy as np 
import mxnet as mx
from tqdm import tqdm_notebook
from mxnet.gluon import nn
from mxnet.gluon.loss import L2Loss
from mxnet import nd, autograd, gluon, init
sys.path.append('../python/')
from CapsuleNet import CapsuleNet, CapsuleMarginLoss
from CapsuleLayer import CapsuleConv, CapsuleDense

In [2]:
mx.random.seed(1)
os.environ['PYTHONUNBUFFERED'] = '1'
os.environ['MXNET_CUDNN_AUTOTUNE_DEFAULT'] = '0'
os.environ['MXNET_ENABLE_GPU_P2P'] = '0'

In [3]:
def transform(data, label):
    return nd.transpose(data.astype(np.float32), (2,0,1))/255, label.astype(np.float32)

## Data for illustration

In [4]:
batch_size = 32
test_batch_size = 128
ctx = mx.cpu(0)
scale_factor = 0.0005
##############################################################
###                    Load Dataset                        ###
##############################################################
train_data = gluon.data.DataLoader(gluon.data.vision.MNIST(train=True,
                                transform=transform),batch_size,
                                shuffle=True)
test_data = gluon.data.DataLoader(gluon.data.vision.MNIST(train=False,
                                transform=transform),
                                test_batch_size,
                                shuffle=False)

for i, (data, label) in enumerate(train_data):
    X = data
    y = label
    if i == 0:
        break

X = X.as_in_context(ctx)
y = y.as_in_context(ctx)

/opt/venv/lib/python3.6/site-packages/mxnet/gluon/data/vision/datasets.py:118: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  label = np.fromstring(fin.read(), dtype=np.uint8).astype(np.int32)
/opt/venv/lib/python3.6/site-packages/mxnet/gluon/data/vision/datasets.py:122: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  data = np.fromstring(fin.read(), dtype=np.uint8)


In [5]:
class CapsuleConv(nn.HybridBlock):
    def __init__(self,dim_vector, out_channels, kernel_size, 
                        strides=1,padding=0):
        """
        dim_vector: 캡슐의 차원을 결정
        out_channels: 서로 다른 capsule type의 갯수
        kernel_size, strides: grid of capsule을 결정
        """
        super(CapsuleConv, self).__init__()
        # 논문에서는 0 - 8
        self.capsules_index = ['dim_'+str(i) for i in range(dim_vector)]
        print('capsule names = {}'.format(self.capsules_index))
        for idx in self.capsules_index:
            setattr(self, idx, nn.Conv2D(out_channels, 
                    kernel_size=kernel_size, strides=strides,
                    padding=padding))

    def squash(self, tensor):
        """Batch Squashing Function
        
        Args:
            tensor : 5-D, (batch_size, num_channel,  height, width, dim_vector)
            
        Return:
            tesnor_squached : 5-D, (batch_size, num_channel, height, width, dim_vector)
        """
        epsilon = 1e-9
        tensor_l2norm = (tensor**2).sum(axis=-1).expand_dims(axis=-1)
        scale_factor = tensor_l2norm / (1 + tensor_l2norm)
        tensor_squashed = tensor * (scale_factor / (tensor_l2norm+epsilon)**0.5 )
        return tensor_squashed
    
    def concact_vectors_in_list(self, vec_list, axis):
        concat_vec = vec_list[0]
        for i in range(1, len(vec_list)):
            concat_vec = nd.concat(concat_vec, vec_list[i], dim=axis)

        return concat_vec
    def hybrid_forward(self,F, X):
                    
        outputs = [getattr(self,idx)(X).expand_dims(axis=-1) for idx in self.capsules_index]

        outputs_cat = self.concact_vectors_in_list(outputs, axis=4)
        outputs_squashed = self.squash(outputs_cat)
        return outputs_squashed


# How to create capsules step by step

## Convolution step (conv1 in paper)

In [6]:
conv1 = nn.HybridSequential()
conv1.add(nn.Conv2D(256,kernel_size=9, strides=1, activation='relu'))
conv1.initialize(ctx =ctx)
conv1_res = conv1(X)
conv1_res.shape

(32, 256, 20, 20)

* Width와 height가 줄어듬: 28 $\rightarrow$ 20 ($\because$ 28 - 9 + 1)
* 32는 Batch size
* 256은 filter의 갯수

## Capsule 생성
* 8개의 원소를 가지는 convolution layer의 list를 만들어놓고, 이를 합하는 과정
* ((6,6,8)이 채널의 갯수만큼) batch size만큼) 있도록 만들어주기 위해서
> NOTE: dimension이 뒤로 가면 갈수록, 가장 최소의 단위라고 생각하면 됨.
> * 8-dimension의 capsule이 
> * $6\times 6$만큼개가 모여서 같은 type의 capsule을 구성하고
> * 32개의 서로 다른 type의 capsule group이 만들어지고
> * 이런 capsule group이 총 32개의 example로 존재

### (32, 32, 6, 6, 1)의 행렬을 8개 만들기

In [7]:
cc = CapsuleConv(8, 32, 9, 2)
cc.capsules_index
cc.initialize(ctx = ctx)
outputs = [getattr(cc,idx)(conv1_res).expand_dims(axis=-1) for idx in cc.capsules_index]
outputs[0].shape

capsule names = ['dim_0', 'dim_1', 'dim_2', 'dim_3', 'dim_4', 'dim_5', 'dim_6', 'dim_7']


(32, 32, 6, 6, 1)

#### NOTE: 256 channel에서 32 channel로 변환하는 convolution 2d에 대해...

In [8]:
cc1 = nn.Conv2D(channels = 32, kernel_size = 9, strides = 2)
cc1.initialize(ctx = ctx)
cc1_res = cc1(conv1_res)

* 다음과 같이 256개의 (9, 9) kernel로 1개의 output channel로 만든 다음
* 이 과정을 32번 반복하는 것
* 그래서, 실제 가지는 서로 다른 weight의 개수는 $9 \times 9 \times 256 \times 32 = 663552$개임

In [9]:
cc1.weight.data

<bound method Parameter.data of Parameter conv9_weight (shape=(32, 256, 9, 9), dtype=<class 'numpy.float32'>)>

* 그래서 얻어지는 결과물의 크기는 다음과 같음

In [10]:
cc1_res.shape

(32, 32, 6, 6)

### (32, 32, 6, 6, 8)의 data로 만들기

* 마지막 dimension을 expand한 후에 마지막 axis에 쌓기

In [11]:
outputs_cat = cc.concact_vectors_in_list(outputs, axis=4)
outputs_cat.shape

(32, 32, 6, 6, 8)

## Squashing

In [12]:
outputs_squashed = cc.squash(outputs_cat)

In [13]:
outputs_cat.shape

(32, 32, 6, 6, 8)

### Squashing하기 전과 후

In [14]:
v_squashed = outputs_squashed.reshape((-1, 8))
v_output = outputs_cat.reshape((-1, 8))

In [15]:
outputs_len = (v_output[0] ** 2).sum(axis = -1)
squashed_len = (v_squashed[0] ** 2).sum(axis = -1)
print('length of output = {}, length of squashed output = {}'\
      .format(outputs_len, squashed_len))

length of output = 
[3.660471]
<NDArray 1 @cpu(0)>, length of squashed output = 
[0.6168993]
<NDArray 1 @cpu(0)>


In [16]:
def length(a):
    return nd.sqrt((a**2).sum(axis = -1))
d1 = outputs_len/length(outputs_len)
d2 = squashed_len/length(squashed_len)
aa = nd.dot(d1, d2)
print('inner product of two vectors = {}'.format(nd.dot(d1, d2).asnumpy()))

inner product of two vectors = [1.]


### Primary capsule looks like...

In [17]:
capsuleConv = nn.HybridSequential()
capsuleConv.add(CapsuleConv(8, 32, 9, 2))
capsuleConv.initialize(ctx = ctx)

capsule names = ['dim_0', 'dim_1', 'dim_2', 'dim_3', 'dim_4', 'dim_5', 'dim_6', 'dim_7']


In [18]:
primary = capsuleConv(conv1_res)

In [19]:
primary.shape

(32, 32, 6, 6, 8)

## Routing by agreement
> maxpooling을 대체하는 개념

* 0은 차원을 그대로 유지하라는 의미
* 1은 차원을 1로 유지하라는 의미
* -1은 차원을 알아서 찾으라는 의미

In [20]:
_primary = primary.reshape((0, -1, 1, 1, 0))
_primary.shape

(32, 1152, 1, 1, 8)

* 이전 단계에는 총 1152개의 capsule이 존재하고
* _primary를 2번 축으로 10배 복사

In [21]:
X_tile = nd.tile(_primary, reps = (1, 1, 10, 1, 1))

### 8차원 capsule에서 16차원 capsule로..
  * 최적의 16차원 capsule을 만들기 위해 routing by agreement 방법을 사용합니다.
  * routing by agreement에는 capsule의 가중치, $c_{ij}$, 결정해야 하는데, 최초의 prior는 0을 사용합니다. (logit값이므로, 같은 확률로 이전 단계의 capsule이 다음 단계의 capsule에 영향을 줌을 의미합니다.)

In [22]:
num_capsules_prev = _primary.shape[1]
batch_size = _primary.shape[0]
out_channels = 10 # output의 class 숫자

routing_weight = nd.random_normal(
                    shape=(1,
                    num_capsules_prev, out_channels,
                    8, 16)
               , name='routing_weight').as_in_context(ctx)

In [23]:
routing_weight.shape

(1, 1152, 10, 8, 16)

In [24]:
W_tile = nd.tile(routing_weight, reps=(batch_size, 1, 1, 1, 1))

In [25]:
linear_combination_3d = nd.batch_dot(
                X_tile.reshape((-1, X_tile.shape[-2], X_tile.shape[-1])), 
                W_tile.reshape((-1, W_tile.shape[-2], W_tile.shape[-1])))
linear_combination = linear_combination_3d.reshape((batch_size,
                                num_capsules_prev, out_channels,
                                1, 16))
# b_ij (1, 1152, 10, 1, 1)
priors = nd.zeros((1, num_capsules_prev, out_channels,1,1), ctx = ctx)

In [26]:
linear_combination.shape

(32, 1152, 10, 1, 16)

In [27]:
priors.shape

(1, 1152, 10, 1, 1)

### Routing by agreement

#### 1번의 iteration만...

In [28]:
softmax_prior = nd.softmax(priors, axis=2) # on num_capsule dimension

In [29]:
softmax_prior.shape

(1, 1152, 10, 1, 1)

In [30]:
softmax_prior.sum(axis = 2)


[[[[1.]]

  [[1.]]

  [[1.]]

  ...

  [[1.]]

  [[1.]]

  [[1.]]]]
<NDArray 1x1152x1x1 @cpu(0)>

In [31]:
output =  softmax_prior * linear_combination

In [32]:
output.shape

(32, 1152, 10, 1, 16)

In [ ]:
output_sum = output.sum(axis=1, keepdims=True) # s_J

#### NOTE: RoutingAlgorithm-line 6

In [ ]:
output_squashed = squash(output_sum) # v_J

#### NOTE: RoutingAlgorithm-line 7

In [ ]:
output_tile = nd.tile(output_squashed, reps=(1,self.num_capsules_prev,1,1,1))
U_times_v = nd.batch_dot(linear_combination.reshape((-1, 1, self.dim_vector)),
                output_tile.reshape((-1, 1, self.dim_vector)),
                transpose_b =True)
U_times_v = U_times_v.reshape((self.batch_size, self.num_capsules_prev,
                self.out_channels, 1, 1))
priors = priors + U_times_v.sum(axis=0).expand_dims(axis=0)

In [8]:
capsule_net.net

HybridSequential(
  (0): HybridSequential(
    (0): Conv2D(1 -> 256, kernel_size=(9, 9), stride=(2, 2))
  )
  (1): HybridSequential(
    (0): CapsuleConv(
      (dim_0): Conv2D(256 -> 32, kernel_size=(9, 9), stride=(2, 2))
      (dim_1): Conv2D(256 -> 32, kernel_size=(9, 9), stride=(2, 2))
      (dim_2): Conv2D(256 -> 32, kernel_size=(9, 9), stride=(2, 2))
      (dim_3): Conv2D(256 -> 32, kernel_size=(9, 9), stride=(2, 2))
      (dim_4): Conv2D(256 -> 32, kernel_size=(9, 9), stride=(2, 2))
      (dim_5): Conv2D(256 -> 32, kernel_size=(9, 9), stride=(2, 2))
      (dim_6): Conv2D(256 -> 32, kernel_size=(9, 9), stride=(2, 2))
      (dim_7): Conv2D(256 -> 32, kernel_size=(9, 9), stride=(2, 2))
    )
  )
  (2): HybridSequential(
    (0): CapsuleDense(
    
    )
  )
  (3): HybridSequential(
    (0): Dense(16 -> 512, Activation(relu))
    (1): Dense(512 -> 1024, Activation(relu))
    (2): Dense(1024 -> 784, Activation(sigmoid))
  )
)

In [5]:
# convert to static graph for speedup
# capsule_net.hybridize()
train(capsule_net, epochs,ctx,train_data,test_data, margin_loss,
            reconstructions_loss, batch_size, scale_factor)

HBox(children=(IntProgress(value=0, max=1875), HTML(value='')))

KeyboardInterrupt: 